<h1><center>Benchmarking with Pauli Decomposition of Continuous-time Quantum Walks</center></h1>

<center> Konstantinos Georgopoulos </center>

In [ ]:
# Import libraries
import time, ast, re
import numpy as np
from numpy import pi as pi
from collections import OrderedDict
import matplotlib.pyplot as plt

# Import Qiskit
from qiskit import Aer, IBMQ
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.monitor import job_monitor
from qiskit.tools.visualization import plot_histogram
from qiskit.extensions import HamiltonianGate

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='', group='', project='') ### Completion needed for running ###
simulator = Aer.get_backend("qasm_simulator")
qc_iterations = 5000 # Iterations per experiment run on the quantum computer
trials = 20 # Number of trials on the quantum computer, each one is run for qc_iterations iterations
sim_iterations = 100000 # Iterations on the simulator (i.e., qc_iterations*trials)
thermal = True # We want thermal decoherence and dephasing to be part of the model. False if not
ideal_counts = {'00': 0, '01': 500, '10': 99000, '11': 500} # From ideal simulation

In [ ]:
%run ctqw_paulidecomp.py
%run QASMParser.py
%run unm.py

In [ ]:
def getExData(exFilename, prob, iterations):
    '''Returns the data as list of dictionaries. prob - True if we want the data to be returned as probability
    distributions instead of counts. NOTE: careful with the name of the datafiles.'''
    path = "/Users/b6035076/Qiskit/qiskit-tutorials-master/PhD Research/Program Benchmarking Quantum Machines/Data/"
    with open(path + exFilename) as f:
        content = f.readlines()
    experimentData = [ast.literal_eval(x.strip('\n')) for x in content]
    f.close()

    # Format the lists with probabilities instead of counts of positions
    if (prob==True):
        for i in range(0,len(experimentData)):
            dct = ast.literal_eval(experimentData[i])
            keys = list(dct.keys())
            values = list(dct.values())
            for j in range(0,len(values)):
                values[j] = values[j]/iterations
            dct = dict(zip(keys, values))
            experimentData[i] = (str(dct))
        
    return experimentData

def getAvgData(data):
    '''Returns the average distribution of all the data collected from the trials on the quantum computer.'''
    avg_dct = dict()
    
    for key in data[0].keys():
        sm = 0
        for i in range(0,len(data)):
            sm = sm + data[i][key]
            
        avg_dct[key] = sm/len(data)
    
    return avg_dct

def getSumData(data):
    '''Returns the cummulative distribution as the sum of the counts for all trials.'''
    sum_dct = dict()
    
    for key in data[0].keys():
        sm = 0
        for i in range (0, len(data)):
            sm = sm + data[i][key]
            
        sum_dct[key] = sm
    
    return sum_dct

# This has to be defined here, cause it will not read the qwQASM() in the python file
def pdctqwNoiseExecute(iterations, thermal, backend, T1s, T2s, graph, gates):
    '''This method executes the circuit. This is necessary, since the circuit has to be generated each time for the new gates to appear'''
    counts = [{}]*iterations
    
    if (thermal == False):
        for i in range(0,iterations):
            sys.stdout.write('\r'+"Simulation count: "+str(i+1))
            circ = pdctqwQASM()
            simulate = execute(circ, backend = Aer.get_backend("qasm_simulator"), shots=1).result()
            counts[i] = simulate.get_counts()
    else:
        noise_thermal = thermalRelaxationChannel(backend, T1s, T2s, graph, gates)
        for i in range(0,iterations):
            sys.stdout.write('\r'+"Simulation count: "+str(i+1))
            circ = pdctqwQASM()
            simulate = execute(circ, backend = Aer.get_backend("qasm_simulator"), basis_gates=noise_thermal.basis_gates, 
                               noise_model=noise_thermal, shots=1).result()
            counts[i] = simulate.get_counts()
    
    return counts

## IBMQ $5$-qubits Bogota Machine

In [ ]:
def pdctqwQASM():
    '''Runs the formatted python file and returns the circuit.'''
    
    %run -i 'QASM/Formatted2qPDCTQWBogota.py'
        
    return circ

def noiseFreepdctqwQASM():
    '''Runs the formatted, noise-free python file and returns the circuit.'''
    
    %run -i 'QASM/NoiseFreeFormatted2qPDCTQWBogota.py'
    
    return circ

# Creating architectural awareness specific for the quantum circuit run
gates = [0, 1]
graph = [[0,1], [1,0]]

# Quantum computer data
datafile = "2qPDCTQWDataBogota.txt" # IMPORTANT: select the correct machine
exData = getExData(datafile, False, qc_iterations)
# print("Quantum computer data:", exData)
sumExData_bogota = getSumData(exData)
print("Sum counts on quantum computer trials for", trials, "trials and", qc_iterations, "iterations each trial:", sumExData_bogota, "\n")

# Preparing the error rates and noise data
path_data = "Data/ibmq_bogota_calibrations_2qPDCTQW.csv"
data = machineData(path_data)
thermal = True
sqRates = getSingleQubitErrorRates(data) # Dictionary containing the single qubit error rates
tqRates = getTwoQubitErrorRates(data) # Dictionary containing the two qubit error rates
measRates = getMeasureErrorRates(data) # Dictionary containing the measurement error rates per qubit
T1s,T2s = getDecoherenceTimes(data)

# Getting information about the quantum circuit
circ = noiseFreepdctqwQASM()
circ_size = circ.size()
circ_depth = circ.depth()

print("\nNumber of gates in the QASM circuit (including measurements):", circ_size) # number of gates in the circuit
# The length of the longest path from the input (or from a preparation) to the output (or a measurement gate), moving forward in time along qubit wires.
print("Depth of the QASM circuit:", circ_depth, "\n")

# Verify the correct machine is in use
device = provider.get_backend('ibmq_bogota')
print("Simulations of the '" + str(device) + "' machine quantum evolution.\n")

# UNM simulations
start_time = time.time()
counts_comb_bogota = pdctqwNoiseExecute(sim_iterations, thermal, device, T1s, T2s, graph, gates)
end_time = time.time()
print("\nTime elapsed:", end_time - start_time, "seconds.")
counts_comb_bogota = getCounts(counts_comb_bogota, sim_iterations)
counts_comb_bogota = dict(OrderedDict(sorted(counts_comb_bogota.items())))
print("\nCounts on noisy quantum simulator:", counts_comb_bogota)

# Get the probabilities of the **ordered** dictionaries
p = getProbabilities(counts_comb_bogota, sim_iterations)
q = getProbabilities(sumExData_bogota, sim_iterations)

# Calculate the HD
h = hellingerDistance(p,q)
print("The HD between the UNM and the Quantum Computer for", sim_iterations, "iterations is:", h)

## IBMQ $5$-qubits Santiago Machine

In [ ]:
def pdctqwQASM():
    '''Runs the formatted python file and returns the circuit.'''
    
    %run -i 'QASM/Formatted2qPDCTQWSantiago.py'
        
    return circ

def noiseFreepdctqwQASM():
    '''Runs the formatted, noise-free python file and returns the circuit.'''
    
    %run -i 'QASM/NoiseFreeFormatted2qPDCTQWSantiago.py'
    
    return circ

# Creating architectural awareness specific for the quantum circuit run
gates = [0, 1]
graph = [[0,1], [1,0]]

# Quantum computer data
datafile = "2qPDCTQWDataSantiago.txt" # IMPORTANT: select the correct machine
exData = getExData(datafile, False, qc_iterations)
# print("Quantum computer data:", exData)
sumExData_santiago = getSumData(exData)
print("\nSum counts on quantum computer trials for", trials, "trials and", qc_iterations, "iterations each trial:", sumExData_santiago, "\n")

# Preparing the error rates and noise data
path_data = "Data/ibmq_santiago_calibrations_2qPDCTQW.csv"
data = machineData(path_data)
thermal = True
sqRates = getSingleQubitErrorRates(data) # Dictionary containing the single qubit error rates
tqRates = getTwoQubitErrorRates(data) # Dictionary containing the two qubit error rates
measRates = getMeasureErrorRates(data) # Dictionary containing the measurement error rates per qubit
T1s,T2s = getDecoherenceTimes(data)

# Getting information about the quantum circuit
circ = noiseFreepdctqwQASM()
circ_size = circ.size()
circ_depth = circ.depth()

print("\nNumber of gates in the QASM circuit (including measurements):", circ_size) # number of gates in the circuit
# The length of the longest path from the input (or from a preparation) to the output (or a measurement gate), moving forward in time along qubit wires.
print("Depth of the QASM circuit:", circ_depth, "\n")

# Verify the correct machine is in use
device = provider.get_backend('ibmq_santiago')
print("Simulations of the '" + str(device) + "' machine quantum evolution.\n")

# UNM simulations
start_time = time.time()
counts_comb_santiago = pdctqwNoiseExecute(sim_iterations, thermal, device, T1s, T2s, graph, gates)
end_time = time.time()
print("\nTime elapsed:", end_time - start_time, "seconds.")
counts_comb_santiago = getCounts(counts_comb_santiago, sim_iterations)
counts_comb_santiago = dict(OrderedDict(sorted(counts_comb_santiago.items())))
print("\nCounts on noisy quantum simulator:", counts_comb_santiago)

# Get the probabilities of the **ordered** dictionaries
p = getProbabilities(countcounts_comb_santiagos_comb, sim_iterations)
q = getProbabilities(sumExData_santiago, sim_iterations)

# Calculate the HD
h = hellingerDistance(p,q)
print("The HD between the UNM and the Quantum Computer for", sim_iterations, "iterations is:", h)

## IBMQ $7$-qubits Casablanca Machine

In [ ]:
def pdctqwQASM():
    '''Runs the formatted python file and returns the circuit.'''
    
    %run -i 'QASM/Formatted2qPDCTQWCasablanca.py'
        
    return circ

def noiseFreepdctqwQASM():
    '''Runs the formatted, noise-free python file and returns the circuit.'''
    
    %run -i 'QASM/NoiseFreeFormatted2qPDCTQWCasablanca.py'
    
    return circ

# Creating architectural awareness specific for the quantum circuit run
gates = [0, 1]
graph = [[0,1], [1,0]]

# Quantum computer data
datafile = "2qPDCTQWDataCasablanca.txt" # IMPORTANT: select the correct machine
exData = getExData(datafile, False, qc_iterations)
# print("Quantum computer data:", exData)
sumExData_casablanca = getSumData(exData)
print("Sum counts on quantum computer trials for", trials, "trials and", qc_iterations, "iterations each trial:", sumExData_casablanca, "\n")

# Preparing the error rates and noise data
path_data = "Data/ibmq_casablanca_calibrations_2qPDCTQW.csv"
data = machineData(path_data)
thermal = True
sqRates = getSingleQubitErrorRates(data) # Dictionary containing the single qubit error rates
tqRates = getTwoQubitErrorRates(data) # Dictionary containing the two qubit error rates
measRates = getMeasureErrorRates(data) # Dictionary containing the measurement error rates per qubit
T1s,T2s = getDecoherenceTimes(data)

# Getting information about the quantum circuit
circ = noiseFreepdctqwQASM()
circ_size = circ.size()
circ_depth = circ.depth()

print("\nNumber of gates in the QASM circuit (including measurements):", circ_size) # number of gates in the circuit
# The length of the longest path from the input (or from a preparation) to the output (or a measurement gate), moving forward in time along qubit wires.
print("Depth of the QASM circuit:", circ_depth, "\n")

# Verify the correct machine is in use
device = provider.get_backend('ibmq_casablanca')
print("Simulations of the '" + str(device) + "' machine quantum evolution.\n")

# UNM simulations
start_time = time.time()
counts_comb_casablanca = pdctqwNoiseExecute(sim_iterations, thermal, device, T1s, T2s, graph, gates)
end_time = time.time()
print("\nTime elapsed:", end_time - start_time, "seconds.")
counts_comb_casablanca = getCounts(counts_comb_casablanca, sim_iterations)
counts_comb_casablanca = dict(OrderedDict(sorted(counts_comb_casablanca.items())))
print("\nCounts on noisy quantum simulator:", counts_comb_casablanca)

# Get the probabilities of the **ordered** dictionaries
p = getProbabilities(counts_comb_casablanca, sim_iterations)
q = getProbabilities(sumExData_casablanca, sim_iterations)

# Calculate the HD
h = hellingerDistance(p,q)
print("The HD between the UNM and the Quantum Computer for", sim_iterations, "iterations is:", h)

# Calculating HD Between Quantum Computer and Ideal Evolution

In [ ]:
h_b = hellingerDistance(getProbabilities(sumExData_bogota, sim_iterations), getProbabilities(ideal_counts, sim_iterations))
h_s = hellingerDistance(getProbabilities(sumExData_santiago, sim_iterations), getProbabilities(ideal_counts, sim_iterations))
h_c = hellingerDistance(getProbabilities(sumExData_casablanca, sim_iterations), getProbabilities(ideal_counts, sim_iterations))
print("HD between the Bogota machine and the ideal distribution:",h_b)
print("HD between the Santiago machine and the ideal distribution:",h_s)
print("HD between the Casablanca machine and the ideal distribution without ancilla:",h_c)

# Calculating HD Between UNM Simulation and Ideal Evolution

In [ ]:
h_b = hellingerDistance(getProbabilities(counts_comb_bogota, sim_iterations), getProbabilities(ideal_counts, sim_iterations))
h_s = hellingerDistance(getProbabilities(counts_comb_santiago, sim_iterations), getProbabilities(ideal_counts, sim_iterations))
h_c = hellingerDistance(getProbabilities(counts_comb_casablanca, sim_iterations), getProbabilities(ideal_counts, sim_iterations))
print("HD between the Bogota machine and the ideal distribution:",h_b)
print("HD between the Santiago machine and the ideal distribution:",h_s)
print("HD between the Casablanca machine and the ideal distribution without ancilla:",h_c)

# Calculating HD Between UNM Simulation and Quantum Computer

In [ ]:
h_b = hellingerDistance(getProbabilities(counts_comb_bogota, sim_iterations), getProbabilities(sumExData_bogota, sim_iterations))
h_s = hellingerDistance(getProbabilities(counts_comb_santiago, sim_iterations), getProbabilities(sumExData_santiago, sim_iterations))
h_c = hellingerDistance(getProbabilities(counts_comb_casablanca, sim_iterations), getProbabilities(sumExData_casablanca, sim_iterations))
print("HD between the Bogota machine and the ideal distribution:",h_b)
print("HD between the Santiago machine and the ideal distribution:",h_s)
print("HD between the Casablanca machine and the ideal distribution without ancilla:",h_c)